In [ ]:

pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.6/599.6 kB 54.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import string
from underthesea import word_tokenize

def softmax(x):
	"""Compute softmax values for each sets of scores in x."""
	e_x = np.exp(x - np.max(x))
	return e_x / e_x.sum()
# tạo lớp w2v
class word2vec(object):
	def __init__(self):
		self.N = 100
		self.X_train = []
		self.y_train = []
		self.window_size = 4
		self.alpha = 0.001
		self.words = []
		self.word_index = {}

	def initialize(self,V,data):
		self.V = V
		self.W = np.random.uniform(-0.8, 0.8, (self.V, self.N))
		self.W1 = np.random.uniform(-0.8, 0.8, (self.N, self.V))
		
		self.words = data
		for i in range(len(data)):
			self.word_index[data[i]] = i

	def feed_forward(self,X):
		self.h = np.dot(self.W.T,X).reshape(self.N,1)
		self.u = np.dot(self.W1.T,self.h)
		#print(self.u)
		self.y = softmax(self.u)
		return self.y
	# cập nhật trọng số cho từ
	def backpropagate(self,x,t):
		e = self.y - np.asarray(t).reshape(self.V,1)
		# e.shape is V x 1
		dLdW1 = np.dot(self.h,e.T)
		X = np.array(x).reshape(self.V,1)
		dLdW = np.dot(X, np.dot(self.W1,e).T)
		self.W1 = self.W1 - self.alpha*dLdW1
		self.W = self.W - self.alpha*dLdW
		# train model
	def train(self,epochs):
		for x in range(1,epochs):	
			self.loss = 0
			for j in range(len(self.X_train)):
				self.feed_forward(self.X_train[j])
				self.backpropagate(self.X_train[j],self.y_train[j])
				C = 0
				for m in range(self.V):
					if(self.y_train[j][m]):
						self.loss += -1*self.u[m][0]
						C += 1
				self.loss += C*np.log(np.sum(np.exp(self.u)))
			print("epoch ",x, " loss = ",self.loss)
			self.alpha *= 1/( (1+self.alpha*x) )
			# dự đoán model
	def predict(self,word,number_of_predictions):
		if word in self.words:
			index = self.word_index[word]
			X = [0 for i in range(self.V)]
			X[index] = 1
			prediction = self.feed_forward(X)
			output = {}
			for i in range(self.V):
				output[prediction[i][0]] = i
			
			top_context_words = []
			for k in sorted(output,reverse=True):
				top_context_words.append(self.words[output[k]])
				if(len(top_context_words)>=number_of_predictions):
					break
	
			return top_context_words
		else:
			print("Word not found in dictionary")
  # xử lý dữ liệu
def preprocessing(corpus):
	training_data = []
	sentences = corpus.split(".")
	for i in range(len(sentences)):
		# Tách từ
		sentence= word_tokenize(sentences[i])
		x = [word.strip(string.punctuation) for word in sentence]
		x = [word.lower() for word in x]
		training_data.append(x)
	return training_data
	

def prepare_data_for_training(sentences,w2v):
	data = {}
	for sentence in sentences:
		for word in sentence:
			if word not in data:
				data[word] = 1
			else:
				data[word] += 1
	V = len(data)
	data = sorted(list(data.keys()))
	vocab = {}
	for i in range(len(data)):
		vocab[data[i]] = i
	
	#for i in range(len(words)):
	for sentence in sentences:
		for i in range(len(sentence)):
			center_word = [0 for x in range(V)]
			center_word[vocab[sentence[i]]] = 1
			context = [0 for x in range(V)]
			
			for j in range(i-w2v.window_size,i+w2v.window_size):
				if i!=j and j>=0 and j<len(sentence):
					context[vocab[sentence[j]]] += 1
			w2v.X_train.append(center_word)
			w2v.y_train.append(context)
	w2v.initialize(V,data)

	return w2v.X_train,w2v.y_train


In [ ]:
corpus='Sông Hàn là một nhánh sông lớn đổ ra biển, có lẽ đây đã là dòng chảy cuối trên hành trình đổ vào biển Đông nên dường như dòng sông chảy chậm lại, lững lờ và uốn lượn quanh co. Sông Hàn rộng dài, đứng ở bên này sông nhìn sang bên kia sông những tòa nhà dường như trở nên bé nhỏ. Đi bộ buổi sáng ven sông Hàn để đón chào bình minh là khoảnh khắc tuyệt vời nhất, khi trời vừa hừng đông, từng gợn sóng lăn tăn bắt đầu nổi lên như sự thức giấc của sông Hàn, mặt nước cũng ửng hồng như còn đang ngái ngủ. Chờ đến khi mặt trời lên, dòng sông như bừng tỉnh, ngàn tia nắng chiếu xuống mặt nước lung linh lấp lánh, và thế là sông Hàn sẽ mặc chiếc áo óng ánh đó suốt cả một ngày dài. Cho đến tận chiều tối, mặt trời chưa kịp lặn thì thành phố đã lên đèn, sông Hàn trở nên sặc sỡ hơn bao giờ hết, chiếc cầu quay với đèn xanh, đỏ, vàng in bóng xuống dòng sông thật lung linh huyền ảo. Vì vẻ đẹp về đêm và khung cảnh lãng mạn của sông Hàn mà ở đây có thêm cây cầu tình yêu, một nơi lý tưởng để tâm sự, bày tỏ yêu thương. Sông Hàn mang một vẻ đẹp thơ mộng, yêu kiều và cũng rất bận rộn, hiếm thấy có con sông nào giữa lòng thành phố lại nhộn nhịp tàu thuyền chở hàng qua lại nhiều như sông Hàn. Đây dường như là một dòng sông không ngủ, từ sáng sớm đến đêm muộn dòng sông dập dìu sóng nước đã tiễn đưa và chào đón biết bao lượt tàu thuyền qua lại, chính vì sự bận rộn của hoạt động vận tải trên sông nên mới có chiếc cầu quay sông Hàn, vào thời điểm ban đêm hoặc lúc có thuyền to cần đi qua cầu, chiếc cầu sẽ quay dọc theo dòng sông để thuyền có thể đi qua, sau đó lại nối lại để hai bên bờ sông có thể đi lại. Sông Hàn đẹp nhất về đêm bởi khi đó nó như một dòng sông chảy giữa ánh sáng, càng tuyệt đẹp hơn nữa là những đêm pháo hoa trên sông Hàn, khi ấy dòng sông Hàn dường như đang thăng hoa, biến mình trở thành một dòng sông ngàn hoa nở rộ đầy sắc màu rực rỡ. Dòng sông Hàn nằm giữa thành phố Đà Nẵng, nơi mà bất kì du khách du lịch nào cũng muốn ghé thăm một lần. Quanh năm, dù là khi nào sông cũng đầy ăm ắp. Nước sông cứ cuồn cuộn, đong đầy như tình cảm nhiệt thành của người dân vùng đất này. Nước sông Hàn không có màu trong vắt như nước mùa thu, hay xanh biếc như bầu trời mùa hạ. Mà lúc nào cũng có màu xanh sẫm huyền bí. Màu xanh ấy, một phần được ánh từ bầu trời bao la phía trên cao, một phần chính bởi vì lòng sông sâu và rộng. Mặt sông Hàn vốn là phẳng lặng. Trông như một tấm gương dành riêng cho những người khổng lồ. Thế nhưng, thật chẳng mấy khi được xem một sông Hàn tĩnh lặng, bởi nơi đây lúc nào cũng tấp nập những chiếc tàu thuyền ngược xuôi với đầy những hàng hóa. Chúng rẽ ra từng cột sóng lớn, bọt nước vùng lên trắng xóa, như vẽ lên chiếc đuôi cá cho con thuyền. Điểm nhấn đẹp nhất của sông Hàn chính là cầu sông Hàn - một cái tên thật mộc mạc và chân chất. Chiếc cầu được thiết kế hình con rồng uốn lượn rất kì công, được xem như một trong những niềm tự hào của người dân Đà Nẵng nói riêng và Việt Nam nói chung. Chính chiếc cầu đã làm tăng thêm vẻ đẹp của con sông. Nhưng đồng thời, đó cũng là một điểm tựa để mọi người có thể quan sát một cách toàn vẹn vẻ đẹp tuyệt vời của con sông ấy. Đặc biệt là vào những buổi đêm. Khi thành phố lên đèn, khi chiếc cầu sáng rực lên với những ánh sáng lấp lánh. Chúng hắt xuống dòng sông, để mặt sông ánh lên muôn vàn màu sắc kì diệu, lung linh như thể đó là dòng sông dẫn đến xứ sở thần tiên.'
epochs = 5
training_data = preprocessing(corpus)
w2v = word2vec()

prepare_data_for_training(training_data,w2v)
w2v.train(epochs)


epoch  1  loss =  31998.15307983864
epoch  2  loss =  31633.64810169326
epoch  3  loss =  31290.53553830176
epoch  4  loss =  30967.538863996546
['lên đèn', 'đã', 'dân', 'muốn', 'tăng', 'phía', 'tàu thuyền', 'đó']


In [ ]:
 # tính độ tương đồng của các từ

import numpy as np

def cosine_similarity(word1, word2, weights, word_index):
    index1 = word_index[word1]
    index2 = word_index[word2]
    
    vector1 = weights[index1]
    vector2 = weights[index2]
    
    dot_product = np.dot(vector1, vector2)
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)
    
    similarity = dot_product / (norm1 * norm2)
    
    return similarity

# Sử dụng ví dụ từ vựng và trọng số từ câu hỏi trướ
word1='màu sắc'
word2='lung linh'
similarity = cosine_similarity(word1, word2, w2v.W, w2v.word_index)
print(f"Độ tương đồng cosine giữa '{word1}' và '{word2}': {similarity}")

Độ tương đồng cosine giữa 'màu sắc' và 'lung linh': 0.01883271633242749


In [ ]:
# lấy embedding từng từ
weights = w2v.W
words=[]
vectors=[]
for word, index in w2v.word_index.items():
    words.append(word)
    vectors.append(weights[index])


In [ ]:
import pandas as pd
words=pd.DataFrame(words)
vectors=pd.DataFrame(vectors)


In [ ]:
words=words.iloc[3:]
vectors=vectors.iloc[3:]

In [ ]:
words.to_csv('words.csv', sep='\t', index=False, header=False)
vectors.to_csv('vectors.csv', sep='\t', index=False, header=False)